In [3]:
import json
# test_path = test_seen or test_unseen
def make_test_data(test_path):
    data_num = 16 if test_path == 'test_seen' else 5
    import json
    from pathlib import Path
    train_list = []
    data_train = []
    for idx in range(1, data_num):
        train_path = Path(f'../../data/{test_path}/dialogues_{idx:0>3}.json')
        data_train.extend(json.load(open(train_path)))

    for i, _ in enumerate(data_train):
        train_dict={}
        train_dict['context']=''
        train_dict['dialogue_id'] = data_train[i]['dialogue_id']
        train_dict['services'] = data_train[i]['services']
        for turn in data_train[i]['turns']:
            train_dict['context']+=turn['utterance']+' '
        train_list.append(train_dict)

    import torch
    schema_path = '../../data/schema.json'
    with open(schema_path, encoding="utf-8") as f:
        schemas = json.load(f)
    schema_dict = {}
    for schema in schemas:
        schema_dict[schema['service_name']]=[f"{schema['service_name']} {slot['name']}: {slot['description']}" for slot in schema['slots']]
    question_list = []
    context_list = []
    id_list = []
    label_list = []
    for i, sample in enumerate(train_list):
        for service in sample['services']:
            service_list = schema_dict[service]
    #                 label[service_list.index(key)] = 1
    #         label_list.extend(label)
            question_list.extend(service_list)

            for slot in service_list:
                context_list.append(sample['context'])
                id_list.append(sample['dialogue_id'])

    test_list = [id_list,question_list,context_list]
    return test_list

In [4]:
test_seen_list = make_test_data('test_seen')
test_unseen_list = make_test_data('test_unseen')

In [5]:
# load dataset
from datasets import Dataset
from datasets.dataset_dict import DatasetDict
test_seen_data = {'idx':[], 'sentence1': [], 'sentence2': []}
test_unseen_data = {'idx':[], 'sentence1': [], 'sentence2': []}
test_seen_data['idx'] = test_seen_list[0]
test_seen_data['sentence1'] = [(test_seen_list[1][i]) for i, _ in enumerate(test_seen_list[0])]
test_seen_data['sentence2'] = [(test_seen_list[2][i]) for i, _ in enumerate(test_seen_list[0])]
test_unseen_data['idx'] = test_unseen_list[0]
test_unseen_data['sentence1'] =[test_unseen_list[1][i] for i, _ in enumerate (test_unseen_list[0])]
test_unseen_data['sentence2'] = [test_unseen_list[2][i] for i, _ in enumerate(test_unseen_list[0])]
test_seen_dataset = DatasetDict({"test":Dataset.from_dict(test_seen_data)})
test_unseen_dataset = DatasetDict({"test":Dataset.from_dict(test_unseen_data)})                             

In [7]:
# load model& tokenizer 
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizer
weights_path = "./weights"
tokenizer_path = "./tokenizer"
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(weights_path, num_labels=2)

device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

OSError: Unable to load weights from pytorch checkpoint file for './weights' at './weights/pytorch_model.bin'If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True. 

In [7]:
# 只有用seen dataset
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    correct = 0
    for i in range(len(labels)):
        if preds[i] == labels[i]:
            correct += 1
    acc = correct / len(labels)
    return {'accuracy': acc}

In [8]:
from transformers import TrainingArguments
learning_rate = 2e-5
batch_size = 10
epochs = 2
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
#     fp16 = True,
#     metric_for_best_model="accuracy",
)

from transformers import Trainer
trainer = Trainer(
    model,
    args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [10]:
import numpy as np
mode = 'test_seen'
dataset  = test_seen_dataset if mode == 'test_seen' else test_unseen_dataset
print(f'encode dataset: {mode}')
encoded_dataset = dataset.map(preprocess_function, batched=True)

print(f'predict mode: {mode}')
preds = trainer.predict(encoded_dataset['test'])
preds_class = np.argmax(preds[0], axis=1)

output_path = f'data/{mode}/{mode}.json'
with open(output_path,"w") as f:
    preds_list = []
    for i, dialogue_id in enumerate(test_seen_list[0]):
        if preds_class[i] != 0 :
            pred_string = ''
            pred_string = f'{dialogue_id}-{test_seen_list[1][i].split()[0]}-{test_seen_list[1][i].split()[1][:-1]}'
            preds_list.append(pred_string)
            print(pred_string)
    json.dump(preds_list, f) 
    print(f'predict finished: data/{mode}/{mode}.json')

encode dataset: test_seen



predict mode: test_seen
PMUL0320-restaurant-restaurant-pricerange
PMUL0320-restaurant-restaurant-area
PMUL0320-restaurant-restaurant-food
PMUL0320-restaurant-restaurant-name
PMUL0320-restaurant-restaurant-bookday
PMUL0320-taxi-taxi-leaveat
PMUL0320-taxi-taxi-destination
PMUL0320-taxi-taxi-departure
PMUL0320-hotel-hotel-pricerange
PMUL0320-hotel-hotel-type
PMUL0320-hotel-hotel-parking
PMUL0320-hotel-hotel-bookday
PMUL0320-hotel-hotel-bookpeople
PMUL0320-hotel-hotel-bookstay
PMUL0320-hotel-hotel-internet
PMUL0320-hotel-hotel-name
MUL2155-train-train-arriveby
MUL2155-train-train-departure
MUL2155-train-train-day
MUL2155-train-train-destination
MUL2155-hotel-hotel-type
MUL2155-hotel-hotel-bookday
MUL2155-hotel-hotel-bookpeople
MUL2155-hotel-hotel-bookstay
MUL2155-hotel-hotel-stars
MUL2155-hotel-hotel-internet
MUL2155-hotel-hotel-name
MUL2155-hotel-hotel-area
PMUL0815-restaurant-restaurant-pricerange
PMUL0815-restaurant-restaurant-food
PMUL0815-restaurant-restaurant-name
PMUL0815-train-tra

SNG01898-train-train-day
SNG01898-train-train-bookpeople
SNG01898-train-train-leaveat
SNG01898-train-train-destination
PMUL4034-train-train-arriveby
PMUL4034-train-train-departure
PMUL4034-train-train-day
PMUL4034-train-train-bookpeople
PMUL4034-train-train-destination
PMUL4034-hotel-hotel-bookday
PMUL4034-hotel-hotel-bookpeople
PMUL4034-hotel-hotel-bookstay
PMUL4034-hotel-hotel-name
PMUL4504-train-train-arriveby
PMUL4504-train-train-departure
PMUL4504-train-train-day
PMUL4504-train-train-destination
PMUL4504-hotel-hotel-pricerange
PMUL4504-hotel-hotel-type
PMUL4504-hotel-hotel-parking
PMUL4504-hotel-hotel-bookday
PMUL4504-hotel-hotel-bookpeople
PMUL4504-hotel-hotel-bookstay
PMUL4504-hotel-hotel-internet
PMUL4504-hotel-hotel-name
PMUL4504-hotel-hotel-area
SNG0678-restaurant-restaurant-area
SNG0678-restaurant-restaurant-food
SNG0678-restaurant-restaurant-name
SNG0678-restaurant-restaurant-bookday
SNG0678-restaurant-restaurant-bookpeople
SNG0678-restaurant-restaurant-booktime
SNG0954-hot

31_00019-Movies_1-location
31_00019-Movies_1-movie_name
31_00024-Services_1-stylist_name
31_00024-Services_1-city
31_00024-Services_1-appointment_date
31_00024-Services_1-appointment_time
31_00024-Movies_1-show_type
31_00024-Movies_1-theater_name
31_00024-Movies_1-show_time
31_00024-Movies_1-show_date
31_00024-Movies_1-location
31_00024-Movies_1-movie_name
31_00026-Services_1-stylist_name
31_00026-Services_1-is_unisex
31_00026-Services_1-city
31_00026-Services_1-appointment_date
31_00026-Services_1-appointment_time
31_00026-Movies_1-theater_name
31_00026-Movies_1-show_time
31_00026-Movies_1-show_date
31_00026-Movies_1-location
31_00026-Movies_1-movie_name
31_00027-Services_1-stylist_name
31_00027-Services_1-is_unisex
31_00027-Services_1-city
31_00027-Services_1-appointment_date
31_00027-Services_1-appointment_time
31_00027-Movies_1-show_type
31_00027-Movies_1-theater_name
31_00027-Movies_1-show_time
31_00027-Movies_1-show_date
31_00027-Movies_1-location
31_00027-Movies_1-movie_name
31_

19_00005-Restaurants_2-category
19_00005-RideSharing_2-destination
19_00005-RideSharing_2-ride_type
19_00005-RideSharing_2-number_of_seats
19_00006-Restaurants_2-restaurant_name
19_00006-Restaurants_2-date
19_00006-Restaurants_2-time
19_00006-Restaurants_2-has_vegetarian_options
19_00006-Restaurants_2-number_of_seats
19_00006-Restaurants_2-location
19_00006-Restaurants_2-category
19_00006-RideSharing_2-destination
19_00006-RideSharing_2-ride_type
19_00006-RideSharing_2-number_of_seats
19_00008-Restaurants_2-restaurant_name
19_00008-Restaurants_2-date
19_00008-Restaurants_2-time
19_00008-Restaurants_2-has_vegetarian_options
19_00008-Restaurants_2-number_of_seats
19_00008-Restaurants_2-price_range
19_00008-Restaurants_2-location
19_00008-Restaurants_2-category
19_00008-RideSharing_2-destination
19_00008-RideSharing_2-ride_type
19_00008-RideSharing_2-number_of_seats
19_00009-Restaurants_2-restaurant_name
19_00009-Restaurants_2-date
19_00009-Restaurants_2-time
19_00009-Restaurants_2-has_ve

13_00036-Hotels_4-location
13_00036-Hotels_4-place_name
13_00037-Events_3-event_name
13_00037-Events_3-date
13_00037-Events_3-number_of_tickets
13_00037-Events_3-city
13_00037-Hotels_4-location
13_00037-Hotels_4-star_rating
13_00037-Hotels_4-place_name
13_00038-Events_3-event_type
13_00038-Events_3-event_name
13_00038-Events_3-date
13_00038-Events_3-number_of_tickets
13_00038-Events_3-city
13_00038-Hotels_4-location
13_00038-Hotels_4-place_name
13_00039-Events_3-event_type
13_00039-Events_3-event_name
13_00039-Events_3-date
13_00039-Events_3-number_of_tickets
13_00039-Events_3-city
13_00039-Hotels_4-location
13_00039-Hotels_4-number_of_rooms
13_00039-Hotels_4-place_name
13_00040-Events_3-event_type
13_00040-Events_3-event_name
13_00040-Events_3-date
13_00040-Events_3-number_of_tickets
13_00040-Events_3-city
13_00040-Hotels_4-location
13_00041-Events_3-event_type
13_00041-Events_3-event_name
13_00041-Events_3-date
13_00041-Events_3-number_of_tickets
13_00041-Events_3-city
13_00041-Hotel

12_00061-Hotels_4-place_name
12_00064-Buses_3-from_city
12_00064-Buses_3-to_city
12_00064-Buses_3-departure_date
12_00064-Buses_3-departure_time
12_00064-Buses_3-additional_luggage
12_00064-Buses_3-num_passengers
12_00064-Buses_3-category
12_00064-Hotels_4-location
12_00064-Hotels_4-number_of_rooms
12_00064-Hotels_4-place_name
12_00064-Hotels_4-smoking_allowed
12_00065-Buses_3-from_city
12_00065-Buses_3-to_city
12_00065-Buses_3-departure_date
12_00065-Buses_3-departure_time
12_00065-Buses_3-additional_luggage
12_00065-Buses_3-num_passengers
12_00065-Buses_3-category
12_00065-Hotels_4-location
12_00065-Hotels_4-place_name
12_00066-Buses_3-from_city
12_00066-Buses_3-to_city
12_00066-Buses_3-departure_date
12_00066-Buses_3-departure_time
12_00066-Buses_3-additional_luggage
12_00066-Buses_3-num_passengers
12_00066-Buses_3-category
12_00066-Hotels_4-location
12_00066-Hotels_4-place_name
12_00067-Buses_3-from_city
12_00067-Buses_3-to_city
12_00067-Buses_3-departure_date
12_00067-Buses_3-depa

In [11]:
import numpy as np
mode = 'test_unseen'
dataset  = test_seen_dataset if mode == 'test_seen' else test_unseen_dataset
print(f'encode dataset: {mode}')
encoded_dataset = dataset.map(preprocess_function, batched=True)

print(f'predict mode: {mode}')
preds = trainer.predict(encoded_dataset['test'])
preds_class = np.argmax(preds[0], axis=1)

output_path = f'../data/{mode}/{mode}.json'
with open(output_path,"w") as f:
    preds_list = []
    for i, dialogue_id in enumerate(test_unseen_list[0]):
        if preds_class[i] != 0 :
            pred_string = ''
            pred_string = f'{dialogue_id}-{test_unseen_list[1][i].split()[0]}-{test_unseen_list[1][i].split()[1][:-1]}'
            preds_list.append(pred_string)
            print(pred_string)
    json.dump(preds_list, f)
    print(f'predict finished: data/{mode}/{mode}.json')

encode dataset: test_unseen



predict mode: test_unseen
1_00029-Flights_3-flight_class
1_00029-Flights_3-origin_city
1_00029-Flights_3-destination_city
1_00029-Flights_3-departure_date
1_00029-Flights_3-return_date
1_00029-Flights_3-outbound_departure_time
1_00029-Flights_3-outbound_arrival_time
1_00029-Flights_3-inbound_arrival_time
1_00029-Flights_3-inbound_departure_time
1_00029-Flights_3-airlines
1_00029-Flights_3-arrives_next_day
1_00030-Flights_3-flight_class
1_00030-Flights_3-origin_city
1_00030-Flights_3-destination_city
1_00030-Flights_3-destination_airport_name
1_00030-Flights_3-departure_date
1_00030-Flights_3-return_date
1_00030-Flights_3-number_stops
1_00030-Flights_3-outbound_departure_time
1_00030-Flights_3-outbound_arrival_time
1_00030-Flights_3-inbound_arrival_time
1_00030-Flights_3-inbound_departure_time
1_00030-Flights_3-airlines
1_00030-Flights_3-arrives_next_day
1_00031-Flights_3-passengers
1_00031-Flights_3-flight_class
1_00031-Flights_3-origin_city
1_00031-Flights_3-destination_city
1_00031-

3_00111-RentalCars_3-pickup_location
3_00111-RentalCars_3-start_date
3_00111-RentalCars_3-pickup_time
3_00111-RentalCars_3-city
3_00111-RentalCars_3-end_date
3_00111-RentalCars_3-add_insurance
3_00112-RentalCars_3-car_type
3_00112-RentalCars_3-pickup_location
3_00112-RentalCars_3-start_date
3_00112-RentalCars_3-pickup_time
3_00112-RentalCars_3-city
3_00112-RentalCars_3-end_date
3_00112-RentalCars_3-add_insurance
3_00113-RentalCars_3-car_type
3_00113-RentalCars_3-pickup_location
3_00113-RentalCars_3-start_date
3_00113-RentalCars_3-pickup_time
3_00113-RentalCars_3-city
3_00113-RentalCars_3-end_date
3_00113-RentalCars_3-add_insurance
3_00114-RentalCars_3-car_type
3_00114-RentalCars_3-pickup_location
3_00114-RentalCars_3-start_date
3_00114-RentalCars_3-pickup_time
3_00114-RentalCars_3-city
3_00114-RentalCars_3-end_date
3_00114-RentalCars_3-add_insurance
3_00115-RentalCars_3-car_type
3_00115-RentalCars_3-pickup_location
3_00115-RentalCars_3-start_date
3_00115-RentalCars_3-pickup_time
3_0011